In [1]:
import numpy as np
import pandas as pd
from astroquery.gaia import Gaia
from astroquery.simbad import Simbad
import matplotlib.pyplot as plt
from astropy.table import Column, join, Table, vstack, hstack
from astropy.io import ascii
from HMXB_pipeline_class import pipeline
import astropy.coordinates as coords
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.visualization import astropy_mpl_style
import corner
from datetime import datetime
import os
from Galactic_traceback import GalacticTraceback

In [2]:
from astropy.io.votable import parse

In [27]:
scoob1 = parse('/home/karan/Documents/UvA/Thesis/scoob1.xml')
ankay = parse('/home/karan/Documents/UvA/Thesis/Tables/Ankay_stars')
NGC6231 = parse('/home/karan/Documents/UvA/Thesis/Tables/NGC_6231')
Cepob1 = parse('/home/karan/Documents/UvA/Thesis/Tables/CEPOB1.xml')
Berkely97 = parse('/home/karan/Documents/UvA/Thesis/DATA/Berkley_97.xml')

In [4]:
Simbad.add_votable_fields("parallax")

In [5]:
options = Simbad.list_votable_fields()

In [6]:
x = Simbad.query_object('Ass Cep OB 1')

In [7]:
x

main_id,ra,dec,coo_err_maj,coo_err_min,coo_err_angle,coo_wavelength,coo_bibcode,plx_bibcode,plx_value,plx_err_prec,plx_prec,plx_qual,plx_err,matched_id
,deg,deg,mas,mas,deg,,,,mas,,,,mas,
object,float64,float64,float32,float32,int16,str1,object,object,float64,int16,int16,str1,float32,object
Ass Cep OB 1,337.25,56.6,--,--,--,,2003AstL...29..311S,,--,--,--,,--,Ass Cep OB 1


In [8]:
def make_astropy_table(parsed_name):
    '''chatgpt this shit fuck simbad'''
    resources = parsed_name.resources
    if len(resources) > 0:
        # Access the first resource
        resource = resources[0]
        
        # Retrieve tables from the resource
        tables = resource.tables
        
        # Access the first table (if there's only one)
        if len(tables) > 0:
            table = tables[0]
            
            # Convert to an Astropy Table
            astro_table = table.to_table()
        else:
            print("No tables found in the first resource.")
    else:
        print("No resources found in the VOTable.")

    return astro_table

In [9]:
from astroquery.simbad import Simbad

def get_gaia_id_from_simbad(object_name):
    '''Query SIMBAD to retrieve the GAIA DR3 ID using query_objectids for a given object name.'''
    simbad = Simbad()

    # Query object identifiers for the object
    result = simbad.query_objectids(object_name)

    if result is not None:
        # Search for the GAIA DR3 ID in the result
        gaia_ids = [obj_id for obj_id in result['id'] if 'Gaia DR3' in obj_id]
        
        if gaia_ids:
            cleaned_gaia_id = gaia_ids[0].replace('Gaia DR3', '').strip()
            return cleaned_gaia_id
            # Return the first GAIA DR3 ID
        else:
            return None
    else:
        return None

# Query for V* V1290 Sco
print(get_gaia_id_from_simbad('V* V1290 Sco'))


5968081461246595456


In [10]:
def add_gaia_ids_to_table(table):
    '''Add GAIA DR3 IDs to an Astropy Table.'''
    gaia_ids = []
    
    # Loop through each object in the table and get the GAIA DR3 ID
    for object_name in table['MAIN_ID']:
        gaia_id = get_gaia_id_from_simbad(object_name)
        gaia_ids.append(gaia_id)
    
    # Add a new column to the table for GAIA DR3 IDs
    table['GAIA_DR3'] = gaia_ids
    return table

In [11]:

# Convert it to an Astropy Table for easy manipulation
scoob1_table = make_astropy_table(scoob1)

In [12]:
scoob1_table

TYPED_ID,MATCHING_ID,ANG_DIST,MAIN_ID,OTYPE_S,RA_d,DEC_d,COO_ERR_MAJA_d,COO_ERR_MINA_d,COO_ERR_ANGLE_d,PMRA,PMDEC,PM_ERR_MAJA,PM_ERR_MINA,PM_ERR_ANGLE,PLX_VALUE,RV_VALUE,Z_VALUE,VLSR,GALDIM_MAJAXIS,GALDIM_MINAXIS,GALDIM_ANGLE,SP_TYPE,MORPH_TYPE,NB_REF,FILTER_NAME_U,FLUX_U,FLUX_ERROR_U,FLUX_SYSTEM_U,FLUX_BIBCODE_U,FLUX_VAR_U,FLUX_MULT_U,FLUX_QUAL_U,FLUX_UNIT_U,FILTER_NAME_B,FLUX_B,FLUX_ERROR_B,FLUX_SYSTEM_B,FLUX_BIBCODE_B,FLUX_VAR_B,FLUX_MULT_B,FLUX_QUAL_B,FLUX_UNIT_B,FILTER_NAME_V,FLUX_V,FLUX_ERROR_V,FLUX_SYSTEM_V,FLUX_BIBCODE_V,FLUX_VAR_V,FLUX_MULT_V,FLUX_QUAL_V,FLUX_UNIT_V,FILTER_NAME_G,FLUX_G,FLUX_ERROR_G,FLUX_SYSTEM_G,FLUX_BIBCODE_G,FLUX_VAR_G,FLUX_MULT_G,FLUX_QUAL_G,FLUX_UNIT_G,FILTER_NAME_R,FLUX_R,FLUX_ERROR_R,FLUX_SYSTEM_R,FLUX_BIBCODE_R,FLUX_VAR_R,FLUX_MULT_R,FLUX_QUAL_R,FLUX_UNIT_R,FILTER_NAME_I,FLUX_I,FLUX_ERROR_I,FLUX_SYSTEM_I,FLUX_BIBCODE_I,FLUX_VAR_I,FLUX_MULT_I,FLUX_QUAL_I,FLUX_UNIT_I,FILTER_NAME_J,FLUX_J,FLUX_ERROR_J,FLUX_SYSTEM_J,FLUX_BIBCODE_J,FLUX_VAR_J,FLUX_MULT_J,FLUX_QUAL_J,FLUX_UNIT_J,FILTER_NAME_H,FLUX_H,FLUX_ERROR_H,FLUX_SYSTEM_H,FLUX_BIBCODE_H,FLUX_VAR_H,FLUX_MULT_H,FLUX_QUAL_H,FLUX_UNIT_H,FILTER_NAME_K,FLUX_K,FLUX_ERROR_K,FLUX_SYSTEM_K,FLUX_BIBCODE_K,FLUX_VAR_K,FLUX_MULT_K,FLUX_QUAL_K,FLUX_UNIT_K,FILTER_NAME_u,FLUX_u,FLUX_ERROR_u,FLUX_SYSTEM_u,FLUX_BIBCODE_u,FLUX_VAR_u,FLUX_MULT_u,FLUX_QUAL_u,FLUX_UNIT_u,FILTER_NAME_g,FLUX_g,FLUX_ERROR_g,FLUX_SYSTEM_g,FLUX_BIBCODE_g,FLUX_VAR_g,FLUX_MULT_g,FLUX_QUAL_g,FLUX_UNIT_g,FILTER_NAME_r,FLUX_r,FLUX_ERROR_r,FLUX_SYSTEM_r,FLUX_BIBCODE_r,FLUX_VAR_r,FLUX_MULT_r,FLUX_QUAL_r,FLUX_UNIT_r,FILTER_NAME_i,FLUX_i,FLUX_ERROR_i,FLUX_SYSTEM_i,FLUX_BIBCODE_i,FLUX_VAR_i,FLUX_MULT_i,FLUX_QUAL_i,FLUX_UNIT_i,FILTER_NAME_z,FLUX_z,FLUX_ERROR_z,FLUX_SYSTEM_z,FLUX_BIBCODE_z,FLUX_VAR_z,FLUX_MULT_z,FLUX_QUAL_z,FLUX_UNIT_z,Diameter_diameter,Diameter_Q,Diameter_unit,Diameter_error,Diameter_filter,Diameter_method,Diameter_bibcode,Distance_distance,Distance_Q,Distance_unit,Distance_merr,Distance_perr,Distance_method,Distance_bibcode,Fe_H_Teff,Fe_H_log_g,Fe_H_Fe_H,Fe_H_flag,Fe_H_CompStar,Fe_H_CatNo,Fe_H_bibcode,Herschel_ObsId,Herschel_alpha,Herschel_delta,PLX_plx,PLX_me,PLX_R,PLX_bibcode,PM_pmra,PM_me_pmra,PM_pmde,PM_me_pmde,PM_system,PM_bibcode,ROT_upVsini,ROT_Vsini,ROT_err,ROT_mes,ROT_qual,ROT_bibcode,V__vartyp,V__Lomax,V__max,V__R_max,V__band,V__Upmin,V__min,V__R_min,V__UpPeriod,V__period,V__R_period,V__epoch,V__R_epoch,V__D_rt,V__R_D_rt,V__bibcode,velocities_type,velocities_Value,velocities_R,velocities_me,velocities_Acc,velocities_Nmes,velocities_nat,velocities_Q,velocities_dom,velocities_res,velocities_d,velocities_Date,velocities_Rem,velocities_Origin,velocities_bibcode,ISO_TDT,ISO_alpha,ISO_delta,IUE_Homogenized_Name,IUE_ComplID,IUE_PROG,IUE_CL,IUE_D,IUE_CAM,IUE_IMAGE,IUE_A,IUE_FES,IUE_MD,IUE_ObsDate,IUE_Time,IUE_ExpTim,IUE_m,IUE_CEB,IUE_S,IUE_Comments,IUE_F,IUE_bibcode,XMM_Obsno,OID4
,,arcsec,,,deg,deg,mas,mas,deg,mas / yr,mas / yr,mas / yr,mas / yr,deg,mas,km / s,,km / s,arcmin,arcmin,deg,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,,,,,,,,,,,,,,unit-degK,cm/s**2,,,,,,,h:m:s,d:m:s,mas,mas,,,mas / yr,mas / yr,mas / yr,mas / yr,,,,km / s,,,,,,,mag,,,,mag,,,day,,day,,,,,,,,,,,,,,,,,,,,,deg,deg,,,,,,,,,,,,,s,,,,,,,,
object,object,float32,object,object,float64,float64,float32,float32,int16,float64,float64,float32,float32,int16,float64,float64,float64,float64,float32,float32,int16,object,object,int32,object,float32,float32,object,object,str1,object,str1,str1,object,float32,float32,object,object,str1,object,str1,str1,object,float32,float32,object,object,str1,object,str1,str1,object,float32,float32,object,object,str1,object,str1,str1,object,float32,float32,object,object,str1,object,str1,str1,object,float32,float32,object,object,str1,object,str1,str1,object,float32,float32,object,object,str1,object,str1,str1,object,float32,float32,object,object,str1,object,str1,str1,

In [13]:
ankay_table = make_astropy_table(ankay)

In [14]:
ankay_table = ankay_table[ankay_table['MAIN_ID'] != 'NAME Sco OB 1']

In [15]:
HMXB_table = ascii.read('HMXB_20241202_.csv',format='csv')

In [16]:
star_170037 =HMXB_table[HMXB_table['Name'] == 'HD 153919']

In [17]:
# sco_ra = scoob1_table['RA_d']
# sco_dec = scoob1_table['DEC_d']

sco_table = scoob1_table['MAIN_ID','RA_d','DEC_d']

In [18]:
ankay_table = add_gaia_ids_to_table(ankay_table)

In [19]:
ankay_table

TYPED_ID,MATCHING_ID,ANG_DIST,MAIN_ID,OTYPE_S,RA_d,DEC_d,COO_ERR_MAJA_d,COO_ERR_MINA_d,COO_ERR_ANGLE_d,PMRA,PMDEC,PM_ERR_MAJA,PM_ERR_MINA,PM_ERR_ANGLE,PLX_VALUE,RV_VALUE,Z_VALUE,VLSR,GALDIM_MAJAXIS,GALDIM_MINAXIS,GALDIM_ANGLE,SP_TYPE,MORPH_TYPE,NB_REF,FILTER_NAME_U,FLUX_U,FLUX_ERROR_U,FLUX_SYSTEM_U,FLUX_BIBCODE_U,FLUX_VAR_U,FLUX_MULT_U,FLUX_QUAL_U,FLUX_UNIT_U,FILTER_NAME_B,FLUX_B,FLUX_ERROR_B,FLUX_SYSTEM_B,FLUX_BIBCODE_B,FLUX_VAR_B,FLUX_MULT_B,FLUX_QUAL_B,FLUX_UNIT_B,FILTER_NAME_V,FLUX_V,FLUX_ERROR_V,FLUX_SYSTEM_V,FLUX_BIBCODE_V,FLUX_VAR_V,FLUX_MULT_V,FLUX_QUAL_V,FLUX_UNIT_V,FILTER_NAME_G,FLUX_G,FLUX_ERROR_G,FLUX_SYSTEM_G,FLUX_BIBCODE_G,FLUX_VAR_G,FLUX_MULT_G,FLUX_QUAL_G,FLUX_UNIT_G,FILTER_NAME_R,FLUX_R,FLUX_ERROR_R,FLUX_SYSTEM_R,FLUX_BIBCODE_R,FLUX_VAR_R,FLUX_MULT_R,FLUX_QUAL_R,FLUX_UNIT_R,FILTER_NAME_I,FLUX_I,FLUX_ERROR_I,FLUX_SYSTEM_I,FLUX_BIBCODE_I,FLUX_VAR_I,FLUX_MULT_I,FLUX_QUAL_I,FLUX_UNIT_I,FILTER_NAME_J,FLUX_J,FLUX_ERROR_J,FLUX_SYSTEM_J,FLUX_BIBCODE_J,FLUX_VAR_J,FLUX_MULT_J,FLUX_QUAL_J,FLUX_UNIT_J,FILTER_NAME_H,FLUX_H,FLUX_ERROR_H,FLUX_SYSTEM_H,FLUX_BIBCODE_H,FLUX_VAR_H,FLUX_MULT_H,FLUX_QUAL_H,FLUX_UNIT_H,FILTER_NAME_K,FLUX_K,FLUX_ERROR_K,FLUX_SYSTEM_K,FLUX_BIBCODE_K,FLUX_VAR_K,FLUX_MULT_K,FLUX_QUAL_K,FLUX_UNIT_K,FILTER_NAME_u,FLUX_u,FLUX_ERROR_u,FLUX_SYSTEM_u,FLUX_BIBCODE_u,FLUX_VAR_u,FLUX_MULT_u,FLUX_QUAL_u,FLUX_UNIT_u,FILTER_NAME_g,FLUX_g,FLUX_ERROR_g,FLUX_SYSTEM_g,FLUX_BIBCODE_g,FLUX_VAR_g,FLUX_MULT_g,FLUX_QUAL_g,FLUX_UNIT_g,FILTER_NAME_r,FLUX_r,FLUX_ERROR_r,FLUX_SYSTEM_r,FLUX_BIBCODE_r,FLUX_VAR_r,FLUX_MULT_r,FLUX_QUAL_r,FLUX_UNIT_r,FILTER_NAME_i,FLUX_i,FLUX_ERROR_i,FLUX_SYSTEM_i,FLUX_BIBCODE_i,FLUX_VAR_i,FLUX_MULT_i,FLUX_QUAL_i,FLUX_UNIT_i,FILTER_NAME_z,FLUX_z,FLUX_ERROR_z,FLUX_SYSTEM_z,FLUX_BIBCODE_z,FLUX_VAR_z,FLUX_MULT_z,FLUX_QUAL_z,FLUX_UNIT_z,Diameter_diameter,Diameter_Q,Diameter_unit,Diameter_error,Diameter_filter,Diameter_method,Diameter_bibcode,Distance_distance,Distance_Q,Distance_unit,Distance_merr,Distance_perr,Distance_method,Distance_bibcode,Fe_H_Teff,Fe_H_log_g,Fe_H_Fe_H,Fe_H_flag,Fe_H_CompStar,Fe_H_CatNo,Fe_H_bibcode,Herschel_ObsId,Herschel_alpha,Herschel_delta,PLX_plx,PLX_me,PLX_R,PLX_bibcode,PM_pmra,PM_me_pmra,PM_pmde,PM_me_pmde,PM_system,PM_bibcode,ROT_upVsini,ROT_Vsini,ROT_err,ROT_mes,ROT_qual,ROT_bibcode,V__vartyp,V__Lomax,V__max,V__R_max,V__band,V__Upmin,V__min,V__R_min,V__UpPeriod,V__period,V__R_period,V__epoch,V__R_epoch,V__D_rt,V__R_D_rt,V__bibcode,velocities_type,velocities_Value,velocities_R,velocities_me,velocities_Acc,velocities_Nmes,velocities_nat,velocities_Q,velocities_dom,velocities_res,velocities_d,velocities_Date,velocities_Rem,velocities_Origin,velocities_bibcode,ISO_TDT,ISO_alpha,ISO_delta,IUE_Homogenized_Name,IUE_ComplID,IUE_PROG,IUE_CL,IUE_D,IUE_CAM,IUE_IMAGE,IUE_A,IUE_FES,IUE_MD,IUE_ObsDate,IUE_Time,IUE_ExpTim,IUE_m,IUE_CEB,IUE_S,IUE_Comments,IUE_F,IUE_bibcode,XMM_Obsno,OID4,GAIA_DR3
,,arcsec,,,deg,deg,mas,mas,deg,mas / yr,mas / yr,mas / yr,mas / yr,deg,mas,km / s,,km / s,arcmin,arcmin,deg,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,,,,,,,,,,,,,,unit-degK,cm/s**2,,,,,,,h:m:s,d:m:s,mas,mas,,,mas / yr,mas / yr,mas / yr,mas / yr,,,,km / s,,,,,,,mag,,,,mag,,,day,,day,,,,,,,,,,,,,,,,,,,,,deg,deg,,,,,,,,,,,,,s,,,,,,,,,
object,object,float32,object,object,float64,float64,float32,float32,int16,float64,float64,float32,float32,int16,float64,float64,float64,float64,float32,float32,int16,object,object,int32,object,float32,float32,object,object,str1,object,str1,str1,object,float32,float32,object,object,str1,object,str1,str1,object,float32,float32,object,object,str1,object,str1,str1,object,float32,float32,object,object,str1,object,str1,str1,object,float32,float32,object,object,str1,object,str1,str1,object,float32,float32,object,object,str1,object,str1,str1,object,float32,float32,object,object,str1,object,str1,str1,object,float32,float32,object,object,str1,object,

In [20]:
NGC_6231_table = add_gaia_ids_to_table(make_astropy_table(NGC6231))

In [21]:
NGC_6231_table

TYPED_ID,MATCHING_ID,ANG_DIST,MAIN_ID,OTYPE_S,RA_d,DEC_d,COO_ERR_MAJA_d,COO_ERR_MINA_d,COO_ERR_ANGLE_d,PMRA,PMDEC,PM_ERR_MAJA,PM_ERR_MINA,PM_ERR_ANGLE,PLX_VALUE,RV_VALUE,Z_VALUE,VLSR,GALDIM_MAJAXIS,GALDIM_MINAXIS,GALDIM_ANGLE,SP_TYPE,MORPH_TYPE,NB_REF,FILTER_NAME_U,FLUX_U,FLUX_ERROR_U,FLUX_SYSTEM_U,FLUX_BIBCODE_U,FLUX_VAR_U,FLUX_MULT_U,FLUX_QUAL_U,FLUX_UNIT_U,FILTER_NAME_B,FLUX_B,FLUX_ERROR_B,FLUX_SYSTEM_B,FLUX_BIBCODE_B,FLUX_VAR_B,FLUX_MULT_B,FLUX_QUAL_B,FLUX_UNIT_B,FILTER_NAME_V,FLUX_V,FLUX_ERROR_V,FLUX_SYSTEM_V,FLUX_BIBCODE_V,FLUX_VAR_V,FLUX_MULT_V,FLUX_QUAL_V,FLUX_UNIT_V,FILTER_NAME_G,FLUX_G,FLUX_ERROR_G,FLUX_SYSTEM_G,FLUX_BIBCODE_G,FLUX_VAR_G,FLUX_MULT_G,FLUX_QUAL_G,FLUX_UNIT_G,FILTER_NAME_R,FLUX_R,FLUX_ERROR_R,FLUX_SYSTEM_R,FLUX_BIBCODE_R,FLUX_VAR_R,FLUX_MULT_R,FLUX_QUAL_R,FLUX_UNIT_R,FILTER_NAME_I,FLUX_I,FLUX_ERROR_I,FLUX_SYSTEM_I,FLUX_BIBCODE_I,FLUX_VAR_I,FLUX_MULT_I,FLUX_QUAL_I,FLUX_UNIT_I,FILTER_NAME_J,FLUX_J,FLUX_ERROR_J,FLUX_SYSTEM_J,FLUX_BIBCODE_J,FLUX_VAR_J,FLUX_MULT_J,FLUX_QUAL_J,FLUX_UNIT_J,FILTER_NAME_H,FLUX_H,FLUX_ERROR_H,FLUX_SYSTEM_H,FLUX_BIBCODE_H,FLUX_VAR_H,FLUX_MULT_H,FLUX_QUAL_H,FLUX_UNIT_H,FILTER_NAME_K,FLUX_K,FLUX_ERROR_K,FLUX_SYSTEM_K,FLUX_BIBCODE_K,FLUX_VAR_K,FLUX_MULT_K,FLUX_QUAL_K,FLUX_UNIT_K,FILTER_NAME_u,FLUX_u,FLUX_ERROR_u,FLUX_SYSTEM_u,FLUX_BIBCODE_u,FLUX_VAR_u,FLUX_MULT_u,FLUX_QUAL_u,FLUX_UNIT_u,FILTER_NAME_g,FLUX_g,FLUX_ERROR_g,FLUX_SYSTEM_g,FLUX_BIBCODE_g,FLUX_VAR_g,FLUX_MULT_g,FLUX_QUAL_g,FLUX_UNIT_g,FILTER_NAME_r,FLUX_r,FLUX_ERROR_r,FLUX_SYSTEM_r,FLUX_BIBCODE_r,FLUX_VAR_r,FLUX_MULT_r,FLUX_QUAL_r,FLUX_UNIT_r,FILTER_NAME_i,FLUX_i,FLUX_ERROR_i,FLUX_SYSTEM_i,FLUX_BIBCODE_i,FLUX_VAR_i,FLUX_MULT_i,FLUX_QUAL_i,FLUX_UNIT_i,FILTER_NAME_z,FLUX_z,FLUX_ERROR_z,FLUX_SYSTEM_z,FLUX_BIBCODE_z,FLUX_VAR_z,FLUX_MULT_z,FLUX_QUAL_z,FLUX_UNIT_z,Diameter_diameter,Diameter_Q,Diameter_unit,Diameter_error,Diameter_filter,Diameter_method,Diameter_bibcode,Distance_distance,Distance_Q,Distance_unit,Distance_merr,Distance_perr,Distance_method,Distance_bibcode,Fe_H_Teff,Fe_H_log_g,Fe_H_Fe_H,Fe_H_flag,Fe_H_CompStar,Fe_H_CatNo,Fe_H_bibcode,Herschel_ObsId,Herschel_alpha,Herschel_delta,PLX_plx,PLX_me,PLX_R,PLX_bibcode,PM_pmra,PM_me_pmra,PM_pmde,PM_me_pmde,PM_system,PM_bibcode,ROT_upVsini,ROT_Vsini,ROT_err,ROT_mes,ROT_qual,ROT_bibcode,V__vartyp,V__Lomax,V__max,V__R_max,V__band,V__Upmin,V__min,V__R_min,V__UpPeriod,V__period,V__R_period,V__epoch,V__R_epoch,V__D_rt,V__R_D_rt,V__bibcode,velocities_type,velocities_Value,velocities_R,velocities_me,velocities_Acc,velocities_Nmes,velocities_nat,velocities_Q,velocities_dom,velocities_res,velocities_d,velocities_Date,velocities_Rem,velocities_Origin,velocities_bibcode,ISO_TDT,ISO_alpha,ISO_delta,IUE_Homogenized_Name,IUE_ComplID,IUE_PROG,IUE_CL,IUE_D,IUE_CAM,IUE_IMAGE,IUE_A,IUE_FES,IUE_MD,IUE_ObsDate,IUE_Time,IUE_ExpTim,IUE_m,IUE_CEB,IUE_S,IUE_Comments,IUE_F,IUE_bibcode,XMM_Obsno,OID4,GAIA_DR3
,,arcsec,,,deg,deg,mas,mas,deg,mas / yr,mas / yr,mas / yr,mas / yr,deg,mas,km / s,,km / s,arcmin,arcmin,deg,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,,,,,,,,,,,,,,unit-degK,cm/s**2,,,,,,,h:m:s,d:m:s,mas,mas,,,mas / yr,mas / yr,mas / yr,mas / yr,,,,km / s,,,,,,,mag,,,,mag,,,day,,day,,,,,,,,,,,,,,,,,,,,,deg,deg,,,,,,,,,,,,,s,,,,,,,,,
object,object,float32,object,object,float64,float64,float32,float32,int16,float64,float64,float32,float32,int16,float64,float64,float64,float64,float32,float32,int16,object,object,int32,object,float32,float32,object,object,str1,object,str1,str1,object,float32,float32,object,object,str1,object,str1,str1,object,float32,float32,object,object,str1,object,str1,str1,object,float32,float32,object,object,str1,object,str1,str1,object,float32,float32,object,object,str1,object,str1,str1,object,float32,float32,object,object,str1,object,str1,str1,object,float32,float32,object,object,str1,object,str1,str1,object,float32,float32,object,object,str1,object,

In [22]:
cepob1_table = make_astropy_table(Cepob1)

In [23]:
cepob1_table = add_gaia_ids_to_table(cepob1_table)

In [24]:
cepob1_table

TYPED_ID,MATCHING_ID,ANG_DIST,MAIN_ID,OTYPE_S,RA_d,DEC_d,COO_ERR_MAJA_d,COO_ERR_MINA_d,COO_ERR_ANGLE_d,PMRA,PMDEC,PM_ERR_MAJA,PM_ERR_MINA,PM_ERR_ANGLE,PLX_VALUE,RV_VALUE,Z_VALUE,VLSR,GALDIM_MAJAXIS,GALDIM_MINAXIS,GALDIM_ANGLE,SP_TYPE,MORPH_TYPE,NB_REF,FILTER_NAME_U,FLUX_U,FLUX_ERROR_U,FLUX_SYSTEM_U,FLUX_BIBCODE_U,FLUX_VAR_U,FLUX_MULT_U,FLUX_QUAL_U,FLUX_UNIT_U,FILTER_NAME_B,FLUX_B,FLUX_ERROR_B,FLUX_SYSTEM_B,FLUX_BIBCODE_B,FLUX_VAR_B,FLUX_MULT_B,FLUX_QUAL_B,FLUX_UNIT_B,FILTER_NAME_V,FLUX_V,FLUX_ERROR_V,FLUX_SYSTEM_V,FLUX_BIBCODE_V,FLUX_VAR_V,FLUX_MULT_V,FLUX_QUAL_V,FLUX_UNIT_V,FILTER_NAME_G,FLUX_G,FLUX_ERROR_G,FLUX_SYSTEM_G,FLUX_BIBCODE_G,FLUX_VAR_G,FLUX_MULT_G,FLUX_QUAL_G,FLUX_UNIT_G,FILTER_NAME_R,FLUX_R,FLUX_ERROR_R,FLUX_SYSTEM_R,FLUX_BIBCODE_R,FLUX_VAR_R,FLUX_MULT_R,FLUX_QUAL_R,FLUX_UNIT_R,FILTER_NAME_I,FLUX_I,FLUX_ERROR_I,FLUX_SYSTEM_I,FLUX_BIBCODE_I,FLUX_VAR_I,FLUX_MULT_I,FLUX_QUAL_I,FLUX_UNIT_I,FILTER_NAME_J,FLUX_J,FLUX_ERROR_J,FLUX_SYSTEM_J,FLUX_BIBCODE_J,FLUX_VAR_J,FLUX_MULT_J,FLUX_QUAL_J,FLUX_UNIT_J,FILTER_NAME_H,FLUX_H,FLUX_ERROR_H,FLUX_SYSTEM_H,FLUX_BIBCODE_H,FLUX_VAR_H,FLUX_MULT_H,FLUX_QUAL_H,FLUX_UNIT_H,FILTER_NAME_K,FLUX_K,FLUX_ERROR_K,FLUX_SYSTEM_K,FLUX_BIBCODE_K,FLUX_VAR_K,FLUX_MULT_K,FLUX_QUAL_K,FLUX_UNIT_K,FILTER_NAME_u,FLUX_u,FLUX_ERROR_u,FLUX_SYSTEM_u,FLUX_BIBCODE_u,FLUX_VAR_u,FLUX_MULT_u,FLUX_QUAL_u,FLUX_UNIT_u,FILTER_NAME_g,FLUX_g,FLUX_ERROR_g,FLUX_SYSTEM_g,FLUX_BIBCODE_g,FLUX_VAR_g,FLUX_MULT_g,FLUX_QUAL_g,FLUX_UNIT_g,FILTER_NAME_r,FLUX_r,FLUX_ERROR_r,FLUX_SYSTEM_r,FLUX_BIBCODE_r,FLUX_VAR_r,FLUX_MULT_r,FLUX_QUAL_r,FLUX_UNIT_r,FILTER_NAME_i,FLUX_i,FLUX_ERROR_i,FLUX_SYSTEM_i,FLUX_BIBCODE_i,FLUX_VAR_i,FLUX_MULT_i,FLUX_QUAL_i,FLUX_UNIT_i,FILTER_NAME_z,FLUX_z,FLUX_ERROR_z,FLUX_SYSTEM_z,FLUX_BIBCODE_z,FLUX_VAR_z,FLUX_MULT_z,FLUX_QUAL_z,FLUX_UNIT_z,Diameter_diameter,Diameter_Q,Diameter_unit,Diameter_error,Diameter_filter,Diameter_method,Diameter_bibcode,Distance_distance,Distance_Q,Distance_unit,Distance_merr,Distance_perr,Distance_method,Distance_bibcode,Fe_H_Teff,Fe_H_log_g,Fe_H_Fe_H,Fe_H_flag,Fe_H_CompStar,Fe_H_CatNo,Fe_H_bibcode,Herschel_ObsId,Herschel_alpha,Herschel_delta,PLX_plx,PLX_me,PLX_R,PLX_bibcode,PM_pmra,PM_me_pmra,PM_pmde,PM_me_pmde,PM_system,PM_bibcode,ROT_upVsini,ROT_Vsini,ROT_err,ROT_mes,ROT_qual,ROT_bibcode,V__vartyp,V__Lomax,V__max,V__R_max,V__band,V__Upmin,V__min,V__R_min,V__UpPeriod,V__period,V__R_period,V__epoch,V__R_epoch,V__D_rt,V__R_D_rt,V__bibcode,velocities_type,velocities_Value,velocities_R,velocities_me,velocities_Acc,velocities_Nmes,velocities_nat,velocities_Q,velocities_dom,velocities_res,velocities_d,velocities_Date,velocities_Rem,velocities_Origin,velocities_bibcode,ISO_TDT,ISO_alpha,ISO_delta,IUE_Homogenized_Name,IUE_ComplID,IUE_PROG,IUE_CL,IUE_D,IUE_CAM,IUE_IMAGE,IUE_A,IUE_FES,IUE_MD,IUE_ObsDate,IUE_Time,IUE_ExpTim,IUE_m,IUE_CEB,IUE_S,IUE_Comments,IUE_F,IUE_bibcode,XMM_Obsno,OID4,GAIA_DR3
,,arcsec,,,deg,deg,mas,mas,deg,mas / yr,mas / yr,mas / yr,mas / yr,deg,mas,km / s,,km / s,arcmin,arcmin,deg,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,mag,,,,,,,,,,,,,,,,,,,,,,unit-degK,cm/s**2,,,,,,,h:m:s,d:m:s,mas,mas,,,mas / yr,mas / yr,mas / yr,mas / yr,,,,km / s,,,,,,,mag,,,,mag,,,day,,day,,,,,,,,,,,,,,,,,,,,,deg,deg,,,,,,,,,,,,,s,,,,,,,,,
object,object,float32,object,object,float64,float64,float32,float32,int16,float64,float64,float32,float32,int16,float64,float64,float64,float64,float32,float32,int16,object,object,int32,object,float32,float32,object,object,str1,object,str1,str1,object,float32,float32,object,object,str1,object,str1,str1,object,float32,float32,object,object,str1,object,str1,str1,object,float32,float32,object,object,str1,object,str1,str1,object,float32,float32,object,object,str1,object,str1,str1,object,float32,float32,object,object,str1,object,str1,str1,object,float32,float32,object,object,str1,object,str1,str1,object,float32,float32,object,object,str1,object,

In [39]:
Berkley_table= make_astropy_table(Berkely97)
Berkley_table = Berkley_table["RA_d", 'DEC_d']
Berk97_sky = SkyCoord(ra=Berkley_table['RA_d'],dec=Berkley_table['DEC_d'])
Berk97_gal = Berk97_sky.transform_to('galactic')
Berkley_table['l'] = Berk97_gal.l
Berkley_table['b'] = Berk97_gal.b
Berkley_table
Berkley_table.write('/home/karan/Documents/UvA/Thesis/DATA/Berkley_97.ecsv',format='ascii.ecsv',overwrite=True)

In [ ]:
def make_Gaia_query(table):
    table = table.to_pandas()
    table_dr3 = table['GAIA_DR3'].dropna()
    list_dr3 = table_dr3.tolist()
    str_dr3 = ','.join(map(str,list_dr3))
    query = f"""
    Select 
    source_id,l,b,ra,dec,parallax,parallax_error
FROM gaiadr3.gaia_source
WHERE source_id IN ({str_dr3})"""
    return query
print(make_Gaia_query(ankay_table))

In [ ]:
scoob1_table = add_gaia_ids_to_table(scoob1_table)

In [ ]:
print(make_Gaia_query(scoob1_table))

In [ ]:
print(ankay_query)

In [ ]:
ra = sco_table['RA_d']
dec = sco_table['DEC_d']

ankay_ra = ankay_table['RA_d']
ankay_dec = ankay_table['DEC_d']
#plt.scatter(ra,dec,s=1,color='k',label='sco')
plt.scatter(ankay_ra,ankay_dec,color='xkcd:pink',label='ankay')
plt.legend()
plt.show()